In [6]:
import psycopg2
import requests
import pandas as pd
import io
import urllib3
import csv
import random

First we need to establish the database connection. If you haven't already, get it running using docker compose

In [7]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432",
)

cur = conn.cursor()

Now to make any requests to the database for data we can use cur.execute()

In [8]:
cur.execute("SELECT * FROM machine_links INNER JOIN features ON machine_links.ccn = features.ccn WHERE cardinality(csv_headers) is NULL") # query to get joined data
res = cur.fetchall()
random.shuffle(res)
df = pd.DataFrame(res)

df = df.iloc[:, 0:9] # remove the double ccn column
df.columns = ["ccn", "state_or_region", "reporting_entity_name_common", "machine_readable_url", "csv_headers", "meets_standard", "bed_count", "zip_code", "medicare_medicaid_eligible"]

df.set_index("ccn", inplace=True) # index by ccn
df

,state_or_region,reporting_entity_name_common,machine_readable_url,csv_headers,meets_standard,bed_count,zip_code,medicare_medicaid_eligible
ccn,,,,,,,,
450747,TX,Palestine Regional Medical Center,https://www.palestineregional.com/sites/palest...,None,True,160,75801,True
241314,MN,Johnson Memorial Hospital,https://jmhsmn.org/wp-content/uploads/2022/02/...,None,True,24,56232,True
110150,GA,Navicent Health Baldwin,https://navicenthealth.org/files/58-2359398_Na...,None,True,140,31061,True
339802,NY,University Hospital Of Brooklyn ( Downstate) T C,https://www.downstate.edu/patient-care/_docume...,None,True,0,11203,True
320006,NM,Eastern New Mexico Medical Center,https://www.enmmc.com/Uploads/Public/Documents...,None,True,149,88201,True
...,...,...,...,...,...,...,...,...
460015,UT,Logan Regional Hospital,https://intermountainhealthcare.org/-/media/fi...,None,True,148,84341,True
190099,LA,Avoyelles Hospital,https://s3.amazonaws.com/ycubaa-production-mar...,None,True,49,71351,True
520189,WI,Aurora Medical Ctr Kenosha,https://www.aurorahealthcare.org/assets/docume...,None,False,74,53142,True


To add some more insight to the provided url, we can parse the file to see which headers are provided by each hospital

In [9]:
def get_headers(f):
    # f: csv file object
    # returns list of csv headers
    f.readline()
    d_reader = csv.DictReader(f)
    headers = d_reader.fieldnames

    return headers


def parse_mrf(url):
    # url: link to machine readable file
    response = requests.get(url)
    print(response.text)
    return get_headers(response)

Scraping occurs here. Results are shuffled to avoid getting stuck on the same errors with a limited amount of time for error handling.

In [10]:
for row in res:
    session = requests.Session()
    retry = urllib3.util.retry.Retry(connect=3, backoff_factor=.5, raise_on_status=False)
    adapter = requests.adapters.HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    ccn = row[0]
    if "scripps" in row[3] or "melissa" in row[3] or "drew" in row[3] or "grandriver" in row[3] or "nuvance" in row[3] or "orangecountyglobalmedicalcenter" in row[3] or "28g1xh366uy0x9ort41hns72" in row[3] or "menifee" in row[3] or "pushmata" in row[3]:
        cur.execute(
            "UPDATE machine_links SET csv_headers=array_append(csv_headers, %s) WHERE ccn=%s", ("skip", str(ccn))
        )
        conn.commit()
        continue
    print(row[3])
    try:
        r = session.get(row[3])
    except:
        continue
    print(r.status_code)
    headers = []
    if str(r.status_code)[0] == str(4) or str(r.status_code)[0] == str(5) or r.headers['Content-Type'] in ["application/pdf", "application/zip", "application/json"]:
        cur.execute(
            "UPDATE machine_links SET csv_headers=array_append(csv_headers, %s) WHERE ccn=%s", (str(r.status_code), str(ccn))
        )
        conn.commit()
        continue
    if row[3][-4:] == "xlsx":
        headers = list(pd.read_excel(r.content, engine='openpyxl').columns)
    elif row[3][-3:] == "xls":
        headers = list(pd.read_excel(r.content, engine="xlrd").columns)
    elif row[3][-3:] == "csv":
        if "sibley" in row[3]:
            headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore")), delimiter="|").columns)
        else:
            headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)
    elif row[3][-3:] == "zip":
        cur.execute(
            "UPDATE machine_links SET csv_headers=array_append(csv_headers, %s) WHERE ccn=%s", (str(r.status_code), str(ccn))
        )
        conn.commit()        
        continue
    cur.execute(
        "UPDATE machine_links SET csv_headers=%s WHERE ccn=%s", (headers, ccn, )
    )
    conn.commit()

https://www.palestineregional.com/sites/palestine/assets/uploads/MRF%20-%20Palestine_Regional_Medical_Center_Standard_Charges_06072021%20742791525.zip
200
https://jmhsmn.org/wp-content/uploads/2022/02/JMHSChargemaster-Master-File-2022.csv
403
https://navicenthealth.org/files/58-2359398_NavicentHealthBaldwin_StandardCharges.json
200
https://www.downstate.edu/patient-care/_documents/charges/146013200-university-hospital-brooklyn-physician-group-standard-charges.xml
200
https://www.enmmc.com/Uploads/Public/Documents/charge-masters/742870118_Eastern%20NM%20Med%20Ctr_standardcharges.csv
200
https://resources.selectmedical.com/Pricing-Transparency/critical-illness-recovery/25-1816237_Select_Specialty_Hospital-Northern_Kentucky_standardcharges.csv
200
https://core.secure.ehc.com/src/util/detail-price-list/82-1635538_hca-houston-med-ctr_standardcharges.csv
200
https://www.texashealth.org/-/media/Project/THR/shared/Documents/PDFs/Business-Office/Pricing/Hospital-Standard-Charges/texas-health-fo

/var/folders/29/818nv1qj7z1c_zb46ggt489m0000gn/T/ipykernel_88191/2157410155.py:35: DtypeWarning: Columns (41,42,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)


404
https://www.emoryhealthcare.org/ui/pricing-transparency/json/2022/110078_emory-university-hospital-midtown_standardcharges.json
200
https://www.wcch.org/sites/wythe/assets/uploads/MRF%20-%20Wythe_County_Community_Standard_Charges_06112021%20202468795.zip
200
https://www.sgmc.org/wp-content/uploads/2022/08/586004467_sgmc_laniercampus_standardcharges_080522.xlsx
200
https://www.tanner.org/upload/docs/PDFs/FinancialAssistance/2021-Georgia-Price-Transparency-website.xlsx
200
https://cdn.upmc.com/-/media/upmc/locations/hospitals/documents/cdm-json-files/machine-readable-csv-not-json/820880337_upmc-carlisle_standardcharges.csv
404
https://resources.selectmedical.com/Pricing-Transparency/critical-illness-recovery/26-3643089_Select_Specialty_Hospital-Akron_standardcharges.csv
200
https://sri.ernesthealth.com/wp-content/uploads/2022/01/SRI_Chargemaster.zip
200
https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/in/743107055_ascension-st-vincent-car

/var/folders/29/818nv1qj7z1c_zb46ggt489m0000gn/T/ipykernel_88191/2157410155.py:35: DtypeWarning: Columns (7,8,9,17,23,25,27,29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)


https://www.wth.org/wp-content/uploads/2022/11/82-5187448_Volunteer_standardcharges.csv
200
https://healthcare.utah.edu/pricing/876000525_UniversityUtahHospital_StandardCharges.csv
404
https://strh.ernesthealth.com/wp-content/uploads/2022/01/STRH_Chargemaster.zip
200
https://oceanshealthcare.com/wp-content/uploads/2022/04/OCEANS-HEALTHCARE_PRICE-TRANSPARENCY_SHOPPABLE-SERVICES_MAR-2022.xlsx
200
https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx?dbName=dbRHARCMTVERNONKY&type=CDMWithoutLabel


...

/var/folders/29/818nv1qj7z1c_zb46ggt489m0000gn/T/ipykernel_88191/2157410155.py:35: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)


200
https://www.healthsourcesaginaw.org/wp-content/uploads/2022/01/386004889_HealthSource-Saginaw_standardcharges.csv
200
https://www.fryemedctr.com/sites/frye/assets/uploads/MRF%20-%20FRYE_REGIONAL_MEDICAL_CENTER_StandardCharges_03072022%20352547114.zip
200
https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/wi/562592868_midwest-orthopedic-specialty-hospital_standardcharges.xlsx
406
https://www.kindredhospitals.com/docs/default-source/default-document-library/locations/transitional-care-hospitals/cms-price-transparency-files/standard-charges-list-2022/522085555_ocala_standarcharges.csv
200
https://healthcare.ascension.org/-/media/project/ascension/healthcare/price-transparency-files/mi/381359180_ascension-borgess-allegan-hospital_standardcharges.xlsx
406
https://apps.para-hcfs.com/PTT/FinalLinks/Reports.aspx?dbName=dbSouthLincolnKemmererWY&type=CDMWithoutLabel
200
https://www.jacksonpurchase.com/sites/jackson/assets/uploads/MRF%20-%20Jackson_

/var/folders/29/818nv1qj7z1c_zb46ggt489m0000gn/T/ipykernel_88191/2157410155.py:35: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,142,143,144,145,146,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,173,174) have mixed types. Specify dtype option on import or set low_memory=False.
  headers = list(pd.read_csv(io.StringIO(r.content.decode('utf-8', errors="ignore"))).columns)


https://www.saintlukeskc.org/sites/default/files/2021-12/440545297_SaintLukesHospitalKC_standardcharges.csv
200


ParserError: Error tokenizing data. C error: Expected 10 fields in line 186, saw 13
